In [2]:
# Importing Required libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from matplotlib.pyplot import figure
import datetime
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from collections import Counter

In [3]:
stop_t_data = pd.read_csv('st.csv',low_memory=False,error_bad_lines=False)

In [4]:
stop_t_data.shape

(1817607, 30)

In [5]:
stop_t_data['st_reason'].unique()

array(['Unknown', 'Doff', 'LATE DOFF', 'Chocking', 'Air Shortage',
       'Count Change', 'Semi-Cleaning', 'Repair - Mechanical',
       'Drum Chocking', 'EmptiesShortage', 'Late Doff', 'Mix Runout',
       'Maintenance - Elect', 'Hand Shortage', 'WRS Chocking',
       'Power Cut', 'WRS Cleaning', 'Miscellaneous', 'Cleaning',
       'UNKNOWN', 'Idle Time', 'Excess Stock', 'Repair - Electrical',
       'Traveler Change', 'Power Interruption', 'Maintenance-Mech'],
      dtype=object)

In [19]:
count_code = '50 CK'
unit_code = 'C'

In [20]:
stop_t_data['st_count'].unique()

array(['0.120', '1.4', '0.0076', '50 CK', '0.13', '0.140', '1.30', '1.2',
       '50s CKC', '60sCKCPREM-S', '60s CK', '0.130', '1.60', '0.0081',
       '50s CHPREM -S', '48sRWT', '50sCKCPREM-P', '55sOPT', '60CW-C',
       '60sRWT', '42sRWT', '55sRWT', '20sRWT', '60s CH5* - P', '48sOPT',
       '55 RWT', '50PST', '60s CH5* - S', '45s CKC', '50s CH5*', '60 CW',
       '42s CKC', '42sOPT', '1.5', '60sCWC-P-SILVER', '40s CKC',
       '60s CWC-S-SILVER', '63sRWT', '60s CWCP-S', '60s CKC', '60s OPT',
       '62s CWC', '42sRWT-36mm', '50s CWC', '20sOPT', '60CWC-P'],
      dtype=object)

In [21]:
stop_data = stop_t_data[['st_date','st_count','st_ucode','st_mcode','st_rcode','st_reason']]

In [22]:
stop_data =  stop_data[stop_data.st_date.str.contains('^2018')]
stop_data = stop_data[stop_data.st_mcode.str.contains('^RF')]
stop_data = stop_data[stop_data['st_count']== count_code]
stop_data = stop_data[stop_data['st_ucode']== unit_code]
stop_data['st_date'] =pd.to_datetime(stop_data.st_date)
stop_data = stop_data.sort_values(by='st_date')
rf_ms = list(stop_data['st_mcode'].unique())
#st=stop_data[(stop_data['st_date'] > '2018-06-01')]
stop_data['st_mcode'].unique()

array(['RF-19', 'RF-37', 'RF-38', 'RF-33', 'RF-43', 'RF-39', 'RF-22',
       'RF-35', 'RF-41', 'RF-34', 'RF-42', 'RF-27', 'RF-28', 'RF-20',
       'RF-47', 'RF-23', 'RF-36', 'RF-21', 'RF-48', 'RF-26', 'RF-46',
       'RF-45', 'RF-44', 'RF-40', 'RF-01', 'RF-31', 'RF-25', 'RF-24',
       'RF-30', 'RF-29', 'RF-32'], dtype=object)

In [23]:
rf_ms

['RF-19',
 'RF-37',
 'RF-38',
 'RF-33',
 'RF-43',
 'RF-39',
 'RF-22',
 'RF-35',
 'RF-41',
 'RF-34',
 'RF-42',
 'RF-27',
 'RF-28',
 'RF-20',
 'RF-47',
 'RF-23',
 'RF-36',
 'RF-21',
 'RF-48',
 'RF-26',
 'RF-46',
 'RF-45',
 'RF-44',
 'RF-40',
 'RF-01',
 'RF-31',
 'RF-25',
 'RF-24',
 'RF-30',
 'RF-29',
 'RF-32']

In [24]:
months = []
for i in range(1,13):
    month = datetime.date(1900,i, 1).strftime('%B')
    months.append(month)
df_t = pd.DataFrame(months, columns = ['months']) 
for ms in rf_ms:
    monthly_data = []
    stop_data_t = stop_data[stop_data['st_mcode'] ==  ms]
    grouped = stop_data_t.groupby('st_date', as_index=False)
    grouped = grouped['st_rcode'].agg(['count'])
    grouped = grouped.reset_index()
    for i in range(1,13):
        mon_grouped_flags = grouped['st_date'].map(lambda x: x.month) == i
        mon_grouped = grouped[mon_grouped_flags]
        monthly_data.append(sum(mon_grouped['count']))
    df_t[ms]=monthly_data
df_t.index = months
df_t.drop(['months'], axis=1, inplace=True)
df_t

,RF-19,RF-37,RF-38,RF-33,RF-43,RF-39,RF-22,RF-35,RF-41,RF-34,...,RF-45,RF-44,RF-40,RF-01,RF-31,RF-25,RF-24,RF-30,RF-29,RF-32
January,374,393,345,361,296,343,379,341,392,370,...,241,251,411,456,0,0,0,0,0,0
February,409,440,389,434,306,394,410,368,335,380,...,267,267,409,286,0,0,0,0,0,0
March,404,432,372,421,73,390,422,346,78,396,...,64,65,421,0,320,310,336,295,281,281
April,479,514,477,474,0,478,472,441,0,465,...,0,0,482,0,450,456,499,444,413,435
May,389,442,391,387,0,413,417,368,0,391,...,0,0,401,0,389,373,392,361,346,373
June,380,157,151,216,0,100,390,232,0,220,...,0,0,94,0,371,133,405,378,364,349
July,373,0,0,0,0,0,388,0,0,0,...,0,0,0,0,434,0,417,398,409,341
August,314,0,0,0,180,0,414,0,159,0,...,171,175,0,0,438,280,413,396,419,30
September,363,0,0,0,416,0,388,0,382,0,...,440,415,0,0,392,381,367,327,389,0
October,242,270,232,0,473,243,434,0,448,0,...,504,514,368,0,433,218,418,431,423,0


In [25]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
data = []
for ms in rf_ms:
    data.append(go.Scatter(
    x=months,
    y=list(df_t[ms]),
    fill='tozeroy',
    mode= 'none',
    name = ms
    ))
    
layout = go.Layout(
    title='Count Code:<b>'+count_code+'</b>',
    xaxis=dict(title='Months in 2018',),
    yaxis=dict(title='Count of Stoppages',)
)

fig = go.Figure(data=data, layout=layout)
plot(fig, filename='Stoppage-Line.html')

'Stoppage-Line.html'

In [11]:
# Reason in a Month Analysis
st_group =  stop_data.groupby(['st_mcode'], as_index=False)
#st_group =  st_group['st_rcode'].agg(['count'])
st_group

In [43]:
st_lst = []
for ms in list(df_t.columns):
    stop_str = []
    monthly_data = []
    df_st = st_group.get_group(ms)
    df_st_group = df_st.groupby(['st_date','st_reason']).agg(['count'])
    df_st_group = df_st_group.reset_index()
    print(ms)
    #print(df_st_group)
    for i in range(1,13):
        st = ''
        dc = dict()
        mon_grouped_flags = df_st_group['st_date'].map(lambda x: x.month) == i
        mon_grouped = df_st_group[mon_grouped_flags]
        #print(mon_grouped)
        for index, row in mon_grouped.iterrows():
            rs = str(row['st_reason'][0])
            cnt= int(row['st_rcode'])
            if rs in list(dc.keys()):
                dc[rs] = dc[rs] + cnt
            else:
                dc[rs] = cnt
        #print(i)
       # print(dc)
        for key in dc.keys():
            st = st + str(key+':'+str(dc[key])+ '\n')
        stop_str.append(st)
    st_lst.append(stop_str)
print(st_lst)

RF-01
RF-02
RF-04
RF-03
[['', 'Doff:187\nLATE DOFF:163\nUnknown:7\n', 'Doff:75\nLATE DOFF:69\nUnknown:2\n', '', '', '', '', '', '', 'Doff:318\nLATE DOFF:219\nUnknown:26\n', 'Doff:163\nLATE DOFF:89\nUnknown:29\n', ''], ['', 'Doff:182\nLATE DOFF:159\nUnknown:5\n', 'Doff:69\nLATE DOFF:60\nUnknown:7\n', '', '', '', '', 'Doff:214\nLATE DOFF:94\nUnknown:10\n', 'Doff:316\nLATE DOFF:182\nUnknown:8\n', 'Doff:385\nLATE DOFF:252\nUnknown:21\n', 'Doff:158\nLATE DOFF:69\nUnknown:10\n', ''], ['', 'Doff:179\nLATE DOFF:143\nUnknown:17\n', 'Doff:70\nLATE DOFF:67\nUnknown:8\n', '', '', '', '', 'Doff:270\nLATE DOFF:146\nUnknown:23\n', 'Doff:314\nLATE DOFF:169\nUnknown:26\n', 'Doff:375\nLATE DOFF:262\nUnknown:50\n', 'Doff:164\nLATE DOFF:98\nUnknown:11\n', ''], ['', '', '', '', '', '', '', 'Doff:220\nLATE DOFF:105\nUnknown:5\n', 'Doff:313\nLATE DOFF:147\nUnknown:12\n', 'Doff:390\nLATE DOFF:234\nUnknown:24\n', 'Doff:166\nLATE DOFF:87\nUnknown:6\n', '']]


In [44]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import itertools
data = []
for (ms,stop_str) in zip(rf_ms,st_lst):
    data.append(go.Scatter(
    x=months,
    y=list(df_t[ms]),
    text = stop_str,
    hoverinfo = 'text',
    fill='tozeroy',
    mode= 'none',
    name = ms
    ))
    
layout = go.Layout(
    title='Count Code:<b>'+count_code+'</b>',
    xaxis=dict(title='Months in 2018',),
    yaxis=dict(title='Count of Stoppages',)
)

fig = go.Figure(data=data, layout=layout)
plot(fig, filename='Stoppage-Line.html')

'Stoppage-Line.html'

In [ ]:
for (a, b, c) in zip(num, color, value): 
     print a, b, c 